# Peparacion del data frame

## Configuración inicial y carga de datos

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)

Cargar archivo

In [3]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    name=fn
df = pd.read_csv(name, sep=";")


Saving Anexo ET_demo_round_traces_2022.csv to Anexo ET_demo_round_traces_2022.csv


Limpiar nulos

In [4]:
df.dropna(how='any', inplace=True)
null_values_new = df.isnull().sum()
print(null_values_new[null_values_new > 0])

Series([], dtype: int64)


Quitar variables con las que no se van a usar en el modelo

In [5]:
droped_vars = ("AbnormalMatch","TravelledDistance","Map","InternalTeamId","MatchId",
               "RoundId","RoundWinner","Survived","RLethalGrenadesThrown","RNonLethalGrenadesThrown",
               "PrimaryAssaultRifle","PrimarySniperRifle","PrimaryHeavy","PrimarySMG","PrimaryPistol",
               "RoundAssists","RoundHeadshots","RoundFlankKills","MatchFlankKills","MatchAssists","MatchHeadshots",
               "MatchKills","Unnamed: 0")

for var in droped_vars:
  df = df.drop(var, axis=1)


transformar objeto a int

In [6]:
# Transformar variables Objecto a int
try:
  df['TimeAlive'] = pd.to_numeric(df['TimeAlive'].str.replace('.', ''))
  df['FirstKillTime'] = pd.to_numeric(df['FirstKillTime'].str.replace('.', ''))
except Exception as e:
  print(f'{e}\n')


Transformar las variables categoricas Team y MatchWinner a binario.

Team:
* Terrorist: 0
* CounterTerrorist: 1

MatchWinner:
* True: 1
* False: 0

In [7]:
df.Team.replace(to_replace=['Terrorist', 'CounterTerrorist'], value=[0, 1], inplace=True )
df.MatchWinner.replace(to_replace=['False', 'True'], value=[0, 1], inplace=True )
df.MatchWinner = df.MatchWinner.astype(int)

In [8]:
df.head()

,Team,MatchWinner,TimeAlive,FirstKillTime,RoundKills,RoundStartingEquipmentValue,TeamStartingEquipmentValue
0,0,1,51120248995704500,0,0,750,4400
1,0,1,4348662552266170,0,0,800,4400
2,0,1,3735469847054540,0,0,1000,4400
3,0,1,4767886136441470,0,0,850,4400
4,0,1,5312251089027310,0,0,1000,4400


## Objetivo

Utilizar las variables para determinar el tiempo de vida (TimeAlive) del jugador

# Regresion Lineal Simple

In [9]:
from sklearn.linear_model import LinearRegression

from sklearn import model_selection

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

In [10]:
seed = 7
validation_size = 0.20

In [11]:
X_vals = ("Team","MatchWinner","FirstKillTime","RoundKills",
          "RoundStartingEquipmentValue","TeamStartingEquipmentValue")

def linearModelScoring(x, y):
  print(f'--- X Value: {x}')

  x = np.array(df[x]).reshape(-1, 1)
  y = np.array(df[y]).reshape(-1, 1)
  Xtrain, Xtest, Ytrain, Ytest = train_test_split(x,y,test_size=0.33)

  model = LinearRegression()

  model.fit(Xtrain, Ytrain)

  Y_hat = model.predict(Xtest)

  mean_squr = mean_squared_error(Ytest, Y_hat)
  mean_abs = mean_absolute_error(Ytest, Y_hat)
  r2 = r2_score(Ytest, Y_hat)

  print(f'Mean Squared Error: {mean_squr}')
  print(f'Mean Absolute Error: {mean_abs}')
  print(f'R2 Score: {r2}')

In [12]:
# scoring based on model_selection
'''
def linearModelScoring(x, y):
  print(f'--- X Value: {x}')

  x = np.array(df[x]).reshape(-1, 1)
  y = np.array(df[y]).reshape(-1, 1)
  model = LinearRegression()

  X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(x, y, test_size=validation_size, random_state=seed)

  kfold = model_selection.KFold(n_splits=10, random_state=seed, shuffle=True)
  cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
  msg = f"%{cv_results.mean()} (%{cv_results.std()})"

  print(msg)
  '''

'\ndef linearModelScoring(x, y):\n  print(f\'--- X Value: {x}\')\n\n  x = np.array(df[x]).reshape(-1, 1)\n  y = np.array(df[y]).reshape(-1, 1)\n  model = LinearRegression()\n\n  X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(x, y, test_size=validation_size, random_state=seed)\n\n  kfold = model_selection.KFold(n_splits=10, random_state=seed, shuffle=True)\n  cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=\'accuracy\')\n  msg = f"%{cv_results.mean()} (%{cv_results.std()})"\n\n  print(msg)\n  '

## Scores

In [29]:
for val in X_vals:
  linearModelScoring(val, "TimeAlive")
  print('')

--- X Value: Team
Mean Squared Error: 2.0758363260893503e+32
Mean Absolute Error: 9670052169057276.0
R2 Score: -3.7494494603995676e-05

--- X Value: MatchWinner
Mean Squared Error: 2.0814393851168777e+32
Mean Absolute Error: 9653322649385564.0
R2 Score: 7.380687393165886e-05

--- X Value: FirstKillTime
Mean Squared Error: 2.062389571350766e+32
Mean Absolute Error: 9640219181989336.0
R2 Score: -5.320805317010624e-06

--- X Value: RoundKills
Mean Squared Error: 2.0680842734158747e+32
Mean Absolute Error: 9649417225966152.0
R2 Score: 0.0015546148583204245

--- X Value: RoundStartingEquipmentValue
Mean Squared Error: 2.0693607585735837e+32
Mean Absolute Error: 9679429522691942.0
R2 Score: 0.001395510792436827

--- X Value: TeamStartingEquipmentValue
Mean Squared Error: 2.053340199091427e+32
Mean Absolute Error: 9613016975513946.0
R2 Score: 0.0009878344495640734



#Regresion Lineal Multiple

In [24]:
x = np.array(df.drop("TimeAlive",axis=1))
y = np.array(df["TimeAlive"])
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x,y,test_size=0.33)

model = LinearRegression()

model.fit(Xtrain, Ytrain)

Y_hat = model.predict(Xtest)

mean_squr = mean_squared_error(Ytest, Y_hat)
mean_abs = mean_absolute_error(Ytest, Y_hat)
r2 = r2_score(Ytest, Y_hat)

print(f'Mean Squared Error: {mean_squr}')
print(f'Mean Absolute Error: {mean_abs}')
print(f'R2 Score: {r2}')

Mean Squared Error: 2.020635716262942e+32
Mean Absolute Error: 9571605918587778.0
R2 Score: 0.0005735093491271437
